In [69]:
import pandas as pd
import re
import numpy as np

In [70]:
colnames=['Name','Date','Time', 'Text','Cleaned_text'] 
exchanges = ['BINANCE', 'BYBIT', 'FTX', 'HUOBI', 'KUCOIN', 'POLONIEX','KRAKEN']
df = pd.read_csv('./1_BitmexCourses.csv',names = colnames,header=None)
print(df['Cleaned_text'][0])
print(df['Text'][0])
df.head()

['LINK', 'LINKUSDT', 'BinanceBuy', 'Between', '7.00', '-', '7.80targets', '8.60', '-', '10', '-', '11', '-', '13', 'Stop', 'Loss', '6.20httpswww.tradingview.comx7IJylz2L', 'BitmexCourses']
#LINK #LINKUSDT (Binance)Buy Between 7.00 - 7.80targets 8.60 - 10 - 11 - 13 Stop Loss 6.20https://www.tradingview.com/x/7IJylz2L/ @BitmexCourses


,Name,Date,Time,Text,Cleaned_text
0,BitmexCourses,2022-09-17,04:41:52,#LINK #LINKUSDT (Binance)Buy Between 7.00 - 7....,"['LINK', 'LINKUSDT', 'BinanceBuy', 'Between', ..."
1,BitmexCourses,2022-09-23,04:37:27,#QNT #QNTUSDT (Binance)Buy Between 100 - 108Ta...,"['QNT', 'QNTUSDT', 'BinanceBuy', 'Between', '1..."
2,BitmexCourses,2022-09-23,05:35:33,#ROSE #ROSEUSDT (Binance)Buy Between 0.055 - 0...,"['ROSE', 'ROSEUSDT', 'BinanceBuy', 'Between', ..."
3,BitmexCourses,2022-09-24,04:44:00,#BNB #BNBUSDT (Binance)Buy Between 270 - 280Ta...,"['BNB', 'BNBUSDT', 'BinanceBuy', 'Between', '2..."
4,BitmexCourses,2022-09-24,09:12:33,#EGLD #EGLDUSDT (Binance)Buy Between 47 - 48.6...,"['EGLD', 'EGLDUSDT', 'BinanceBuy', 'Between', ..."


In [71]:
exchanges = 'Binance|kucoin|bitbuy'
conversion = 'usdt|usd|btc|eth'
exchangeregex = '|'.join(exchanges)
target = ('|').join(['1','2','3'])
def extractor(i):
  text = re.sub('[\',\[\]]', '', i)
  coin_name,conversion_name,call_exchange,call_type,call_duration,leverage,Targets,stop_loss= ["None"]*8
  if(re.search('stop\s*loss',text,re.IGNORECASE) and re.search('target|targets',text,re.IGNORECASE)):    

    coin_name=re.findall('(\w+)(?:usdt|usd|btc|eth)',text,re.IGNORECASE)

    conversion_name = re.findall('\w+({})'.format(conversion),str(text),re.IGNORECASE)

    call_exchange = re.findall('Binance|kucoin|bitbuy'.format(exchanges),text,re.IGNORECASE)

    call_nature = re.findall('sell|buy',text,re.IGNORECASE)

    """duration not used """
    call_duration = re.findall('long|short',text,re.IGNORECASE)

    """some targets are not extracted although format is same """
    Targets = re.findall("Target[s]*[ ]*(.*?)[a-zA-Z]",text,re.IGNORECASE)
    t = re.sub(' ','',Targets[0])
    Targets = t.split('-')


    leverage = re.findall('[ ]*[-]*[ ]*(\d+)x',text,re.IGNORECASE)

    """ stop loss not used that much"""
    stop_loss = re.findall('stop\s*loss\s*-*\s*(\d.\d*)',text,re.IGNORECASE)

    
  print( "coin_name,conversion_name,call_exchange,call_type,call_duration,leverage,Targets,stop_loss")
  return {
      "status": "A",
      "Username": "Andrew Tate",
      "Date": "2022-09-16",
      "Time": "15:31:42",
      "Coin": coin_name,
      "Type": call_duration,
      "Exchange": call_exchange,
      "Conversion": conversion_name,
      "Targets": Targets,
      "Stoploss": stop_loss,
      "Leverage": leverage,
      "Nature": call_nature,
      "Range": [31899.35, 320000],
    }
  
df["Call"] = df['Cleaned_text'].apply(extractor)

In [72]:
def func(x):
    for i in range(len(x)):
        if (len(x[i])==1):
            x[i]=x[i][0]
        elif not (x[i]):
            x[i]="None"
    return x
df["Call"] = df["Call"].apply(func)

In [73]:
df = df.drop(columns=["Text"])
pd.to_pickle(df,"2_BitmexCourses.pkl")